In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("creditcard.csv")

# Quick check
print(df.head())
print(df['Class'].value_counts())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [2]:
X = df.drop(columns=["Class"])
y = df["Class"]

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [4]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [5]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_resampled, y_resampled)

RandomForestClassifier(random_state=42)

In [6]:
from sklearn.linear_model import LogisticRegression

# Initialize Logistic Regression
lr_model = LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)

# Train model
lr_model.fit(X_resampled, y_resampled)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [7]:
from xgboost import XGBClassifier

# Initialize XGBoost
xgb_model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train model
xgb_model.fit(X_resampled, y_resampled)


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [17:38:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [8]:
from lightgbm import LGBMClassifier

# Initialize LightGBM
lgbm_model = LGBMClassifier(n_estimators=100, random_state=42)

# Train model
lgbm_model.fit(X_resampled, y_resampled)


[LightGBM] [Info] Number of positive: 227451, number of negative: 227451
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.174954 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 454902, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


LGBMClassifier(random_state=42)

In [9]:
import joblib

joblib.dump(rf_model, "rf_model.pkl")      # Random Forest
joblib.dump(lr_model, "lr_model.pkl")      # Logistic Regression
joblib.dump(xgb_model, "xgb_model.pkl")    # XGBoost
joblib.dump(lgbm_model, "lgbm_model.pkl")  # LightGBM

['lgbm_model.pkl']

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "XGBoost": xgb_model,
    "LightGBM": lgbm_model
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"\n{name} Evaluation:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Random Forest Evaluation:
Accuracy: 0.999403110845827
Precision: 0.826530612244898
Recall: 0.826530612244898
F1 Score: 0.826530612244898
ROC AUC: 0.9131158266626853
Confusion Matrix:
 [[56847    17]
 [   17    81]]

Logistic Regression Evaluation:
Accuracy: 0.9800919911519961
Precision: 0.07331136738056013
Recall: 0.9081632653061225
F1 Score: 0.13567073170731708
ROC AUC: 0.944189609580467
Confusion Matrix:
 [[55739  1125]
 [    9    89]]

XGBoost Evaluation:
Accuracy: 0.9993504441557529
Precision: 0.7904761904761904
Recall: 0.8469387755102041
F1 Score: 0.8177339901477833
ROC AUC: 0.9232759437483491
Confusion Matrix:
 [[56842    22]
 [   15    83]]

LightGBM Evaluation:
Accuracy: 0.998735999438222
Precision: 0.5984848484848485
Recall: 0.8061224489795918
F1 Score: 0.6869565217391305
ROC AUC: 0.9025952002917093
Confusion Matrix:
 [[56811    53]
 [   19    79]]


In [12]:
import numpy as np
import joblib

# Load your trained model
model = joblib.load("xgb_model.pkl")  # Change file name if needed

# Example legitimate transaction (Time, V1, ..., V28, Amount)
legit_example = np.array([
    50000.0, 0.1, -0.3, 0.2, 0.5, -0.4, 0.3, 0.1, 0.0, -0.2,
    0.4, -0.1, 0.3, -0.5, 0.2, 0.6, -0.3, 0.1, -0.4, 0.3,
    0.0, 0.5, -0.2, 0.1, -0.1, 0.4, -0.3, 0.2, 0.1, 100.0
]).reshape(1, -1)

# Example fraudulent transaction (Time, V1, ..., V28, Amount)
fraud_example = np.array([
    150000.0, -5.1, 8.2, -3.4, 7.1, -6.2, 5.8, -4.9, 4.3, -2.1,
    3.9, -4.5, 6.7, -7.3, 8.5, -9.1, 7.8, -8.2, 6.5, -7.4,
    8.1, -6.9, 7.2, -5.8, 6.9, -4.3, 5.6, -3.9, 4.2, 2500.0
]).reshape(1, -1)

# Predict results
legit_pred = model.predict(legit_example)[0]
fraud_pred = model.predict(fraud_example)[0]

# Predict probabilities
legit_prob = model.predict_proba(legit_example)[0][1]
fraud_prob = model.predict_proba(fraud_example)[0][1]

print("Legitimate Prediction:", "Fraud" if legit_pred == 1 else "Legitimate")
print(f"Fraud Probability: {legit_prob:.4f}")
print("Fraudulent Prediction:", "Fraud" if fraud_pred == 1 else "Legitimate")
print(f"Fraud Probability: {fraud_prob:.4f}")

Legitimate Prediction: Legitimate
Fraud Probability: 0.0001
Fraudulent Prediction: Legitimate
Fraud Probability: 0.0000


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("creditcard.csv")
X = df.drop(columns=["Class", "Time"])  # Remove target + time
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [14]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [15]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=200, random_state=42, eval_metric='logloss')
model.fit(X_resampled, y_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [16]:
import joblib
joblib.dump(model, "fraud_model.pkl")

['fraud_model.pkl']

In [17]:
import joblib
import numpy as np

# Load model
model = joblib.load("fraud_model.pkl")

# Example legitimate transaction (V1–V28 + Amount)
legit_example = np.array([[-1.35980713, -0.07278117, 2.53634674, 1.37815522, -0.33832077,
                           0.46238778, 0.23959855, 0.0986979, 0.36378697, 0.09079417,
                           -0.55159953, -0.61780086, -0.99138985, -0.31116935, 1.46817697,
                           -0.47040053, 0.20797124, 0.02579058, 0.40399296, 0.2514121,
                           -0.01830678, 0.27783758, -0.11047391, 0.06692807, 0.12853936,
                           -0.18911484, 0.13355838, -0.02105305, 149.62]])

# Example fraudulent transaction (V1–V28 + Amount)
fraud_example = np.array([[1.19185711, 0.26615071, 0.16648011, 0.44815408, 0.06001765,
                           -0.08236081, -0.07880298, 0.08510165, -0.25542513, -0.16697441,
                           1.61272666, 1.06523531, 0.48909502, -0.1437723, 0.63555809,
                           0.46391704, -0.11480466, -0.18336127, -0.14578304, -0.06908314,
                           -0.22577525, -0.63867195, 0.10128802, -0.33984648, 0.1671704,
                           0.12589453, -0.0089831, 0.01472417, 0.0]])

# Predict Legitimate
legit_prob = model.predict_proba(legit_example)[0][1]
print("Legitimate Prediction:", "Fraud" if legit_prob > 0.5 else "Legitimate", "Probability:", legit_prob)

# Predict Fraudulent
fraud_prob = model.predict_proba(fraud_example)[0][1]
print("Fraudulent Prediction:", "Fraud" if fraud_prob > 0.5 else "Legitimate", "Probability:", fraud_prob)

Legitimate Prediction: Legitimate Probability: 1.7680555e-05
Fraudulent Prediction: Legitimate Probability: 2.8970799e-06


In [18]:
# Step 1: Install dependencies
!pip install imbalanced-learn xgboost joblib

# Step 2: Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import joblib

# Step 3: Load Dataset
df = pd.read_csv("creditcard.csv")  # Change path if needed

# Step 4: Features & Target
X = df.drop(columns=["Class", "Time"])  # Removing Time
y = df["Class"]

# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 6: Apply SMOTE to handle imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print("Original dataset shape:", y_train.value_counts())
print("Resampled dataset shape:", y_resampled.value_counts())

# Step 7: Train Random Forest
rf1_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf1_model.fit(X_resampled, y_resampled)

# Step 8: Train XGBoost
xgb1_model = XGBClassifier(eval_metric='logloss', random_state=42, use_label_encoder=False)
xgb1_model.fit(X_resampled, y_resampled)

# Step 9: Evaluation function
def evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    print(f"\n{name} Evaluation:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_prob))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Step 10: Evaluate both models
evaluate_model("Random Forest", rf1_model, X_test, y_test)
evaluate_model("XGBoost", xgb1_model, X_test, y_test)

# Step 11: Save the best model (Choose the one with highest Recall + Precision balance)
joblib.dump(rf1_model, "fraud_model1.pkl")  # Save RF model
# joblib.dump(xgb_model, "fraud_model.pkl")  # Uncomment to save XGBoost instead
print("\n✅ Model saved as fraud_mode1l.pkl")

Original dataset shape: Class
0    227451
1       394
Name: count, dtype: int64
Resampled dataset shape: Class
0    227451
1    227451
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [18:07:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Random Forest Evaluation:
Accuracy: 0.9994733330992591
Precision: 0.8469387755102041
Recall: 0.8469387755102041
F1 Score: 0.8469387755102041
ROC AUC: 0.9680177659837148
Confusion Matrix:
 [[56849    15]
 [   15    83]]

XGBoost Evaluation:
Accuracy: 0.999420666409185
Precision: 0.8037383177570093
Recall: 0.8775510204081632
F1 Score: 0.8390243902439024
ROC AUC: 0.9805504433061913
Confusion Matrix:
 [[56843    21]
 [   12    86]]

✅ Model saved as fraud_mode1l.pkl


In [20]:
import joblib
import numpy as np

# Load the trained model
model = joblib.load("fraud_model1.pkl")  # <-- Make sure filename is correct

# Legitimate example (V1–V28 + Amount)
legit_example = np.array([
    -1.35980713, -0.07278117, 2.53634674, 1.37815522, -0.33832077,
    0.46238778, 0.23959855, 0.0986979, 0.36378697, 0.09079417,
    -0.55159953, -0.61780086, -0.99138985, -0.31116935, 1.46817697,
    -0.47040053, 0.20797124, 0.02579058, 0.40399296, 0.2514121,
    -0.01830678, 0.27783758, -0.11047391, 0.06692807, 0.12853936,
    -0.18911484, 0.13355838, -0.02105305, 149.62
]).reshape(1, -1)

# Fraudulent example (V1–V28 + Amount)
fraud_example = np.array([
    1.19185711, 0.26615071, 0.16648011, 0.44815408, 0.06001765,
    -0.08236081, -0.07880298, 0.08510165, -0.25542513, -0.16697441,
    1.61272666, 1.06523531, 0.48909502, -0.1437723, 0.63555809,
    0.46391704, -0.11480466, -0.18336127, -0.14578304, -0.06908314,
    -0.22577525, -0.63867195, 0.10128802, -0.33984648, 0.1671704,
    0.12589453, -0.008983099, 0.01472417, 0.0
]).reshape(1, -1)

# Predictions
for name, example in [("Legitimate", legit_example), ("Fraudulent", fraud_example)]:
    pred = model.predict(example)[0]
    prob = model.predict_proba(example)[0][1]
    print(f"{name} Prediction: {'Fraudulent' if pred == 1 else 'Legitimate'} | Probability: {prob:.4f}")

Legitimate Prediction: Legitimate | Probability: 0.0000
Fraudulent Prediction: Legitimate | Probability: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [21]:
import joblib

joblib.dump(xgb_model, "fraud_model_xgb.pkl")
print("✅ XGBoost model saved as fraud_model_xgb.pkl")

✅ XGBoost model saved as fraud_model_xgb.pkl


In [24]:
import joblib
import numpy as np

# Load the trained model
model = joblib.load("fraud_model_xgb.pkl")

# ✅ Example 1: Legitimate transaction (Time + V1 to V28 + Amount)
legit_example = np.array([
    0.0,  # Time
    -1.35980713, -0.07278117, 2.53634674, 1.37815522, -0.33832077,
    0.46238778, 0.23959855, 0.0986979, 0.36378697, 0.09079417,
    -0.55159953, -0.61780086, -0.99138985, -0.31116935, 1.46817697,
    -0.47040053, 0.20797124, 0.02579058, 0.40399296, 0.2514121,
    -0.01830678, 0.27783758, -0.11047391, 0.06692807, 0.12853936,
    -0.18911484, 0.13355838, -0.02105305, 149.62
]).reshape(1, -1)

# ✅ Example 2: Fraudulent transaction (Time + V1 to V28 + Amount)
fraud_example = np.array([
    0.0,  # Time
    1.19185711, 0.26615071, 0.16648011, 0.44815408, 0.06001765,
    -0.08236081, -0.07880298, 0.08510165, -0.25542513, -0.16697441,
    1.61272666, 1.06523531, 0.48909502, -0.1437723, 0.63555809,
    0.46391704, -0.11480466, -0.18336127, -0.14578304, -0.06908314,
    -0.22577525, -0.63867195, 0.10128802, -0.33984648, 0.1671704,
    0.12589453, -0.008983099, 0.01472417, 0.0
]).reshape(1, -1)

# ✅ Test predictions
for name, example in [("Legitimate", legit_example), ("Fraudulent", fraud_example)]:
    pred = model.predict(example)[0]
    prob = model.predict_proba(example)[0][1]  # Probability of Fraud
    print(f"{name} Prediction: {'Fraudulent' if pred == 1 else 'Legitimate'} | Probability: {prob:.4f}")

Legitimate Prediction: Legitimate | Probability: 0.0000
Fraudulent Prediction: Legitimate | Probability: 0.0000


In [25]:
# 1️⃣ Install dependencies (only if not installed)
!pip install imbalanced-learn xgboost joblib

# 2️⃣ Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import joblib

# 3️⃣ Load dataset
df = pd.read_csv("creditcard.csv")  # change to your dataset path

# Drop Time column if it exists
if 'Time' in df.columns:
    df = df.drop(columns=['Time'])

# Separate features and target
X = df.drop(columns=['Class'])
y = df['Class']

# 4️⃣ Train-test split (stratified to balance classes in split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 5️⃣ Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 6️⃣ Train XGBoost with imbalance handling
scale_pos_weight = y_resampled.value_counts()[0] / y_resampled.value_counts()[1]

xgb_model2 = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    scale_pos_weight=scale_pos_weight,
    eval_metric="aucpr",
    random_state=42,
    use_label_encoder=False
)

xgb_model2.fit(X_resampled, y_resampled)

# 7️⃣ Evaluate
y_pred = xgb_model2.predict(X_test)
y_prob = xgb_model2.predict_proba(X_test)[:, 1]

print("\nXGBoost Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob)}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# 8️⃣ Save model
joblib.dump(xgb_model2, "fraud_model_xgb1.pkl")
print("\n✅ Model saved as fraud_model_xgb.pkl")

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [18:20:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBoost Evaluation:
Accuracy: 0.9994382219725431
Precision: 0.8113207547169812
Recall: 0.8775510204081632
F1 Score: 0.8431372549019608
ROC AUC: 0.9830623442398906
Confusion Matrix:
 [[56844    20]
 [   12    86]]

✅ Model saved as fraud_model_xgb.pkl


In [26]:
import joblib
import numpy as np

# Load model
model = joblib.load("fraud_model_xgb1.pkl")

# Example values (V1–V28 + Amount)
# Replace with real ones from your dataset for better accuracy

# Legitimate transaction example
legit_example = np.array([
    -1.35980713, -0.07278117,  2.53634674,  1.37815522, -0.33832077,
    0.46238778,  0.23959855,  0.0986979,  0.36378697,  0.09079417,
    -0.55159953, -0.61780086, -0.99138985, -0.31116935,  1.46817697,
    -0.47040053,  0.20797124,  0.02579058,  0.40399296,  0.2514121,
    -0.01830678,  0.27783758, -0.11047391,  0.06692807,  0.12853936,
    -0.18911484,  0.13355838, -0.02105305, 149.62
]).reshape(1, -1)

# Fraudulent transaction example
fraud_example = np.array([
    1.19185711, 0.26615071, 0.16648011, 0.44815408, 0.06001765,
    -0.08236081, -0.07880298, 0.08510165, -0.25542513, -0.16697441,
    1.61272666, 1.06523531, 0.48909502, -0.1437723, 0.63555809,
    0.46391704, -0.11480466, -0.18336127, -0.14578304, -0.06908314,
    -0.22577525, -0.63867195, 0.10128802, -0.33984648, 0.1671704,
    0.12589453, -0.008983099, 0.01472417, 2.69
]).reshape(1, -1)

# Predictions
for name, example in [("Legitimate", legit_example), ("Fraudulent", fraud_example)]:
    pred = model.predict(example)[0]
    prob = model.predict_proba(example)[0][1]
    print(f"{name} Prediction: {'Fraudulent' if pred == 1 else 'Legitimate'} | Probability: {prob:.4f}")


Legitimate Prediction: Legitimate | Probability: 0.0001
Fraudulent Prediction: Legitimate | Probability: 0.0001


In [27]:
import pandas as pd

# Load your preprocessed dataset (same one you used for training)
df = pd.read_csv("creditcard.csv")

# Fraudulent example (Class = 1)
fraud_example = df[df['Class'] == 1].iloc[0, :-1].values  # all features except Class
print("Fraudulent Example Values:")
print(fraud_example.tolist())

# Legitimate example (Class = 0)
legit_example = df[df['Class'] == 0].iloc[0, :-1].values
print("\nLegitimate Example Values:")
print(legit_example.tolist())

Fraudulent Example Values:
[406.0, -2.3122265423263, 1.95199201064158, -1.60985073229769, 3.9979055875468, -0.522187864667764, -1.42654531920595, -2.53738730624579, 1.39165724829804, -2.77008927719433, -2.77227214465915, 3.20203320709635, -2.89990738849473, -0.595221881324605, -4.28925378244217, 0.389724120274487, -1.14074717980657, -2.83005567450437, -0.0168224681808257, 0.416955705037907, 0.126910559061474, 0.517232370861764, -0.0350493686052974, -0.465211076182388, 0.320198198514526, 0.0445191674731724, 0.177839798284401, 0.261145002567677, -0.143275874698919, 0.0]

Legitimate Example Values:
[0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.0

In [29]:
import joblib
import numpy as np

# Load the trained model
model = joblib.load("fraud_model_xgb1.pkl")

# Legitimate transaction example (remove Time)
legit_example = [
    -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443,
    -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507,
    0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348,
    -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478,
    0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705,
    -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731,
    0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215,
    149.62
]

# Fraudulent transaction example (remove Time)
fraud_example = [
    -2.3122265423263, 1.95199201064158, -1.60985073229769, 3.9979055875468,
    -0.522187864667764, -1.42654531920595, -2.53738730624579, 1.39165724829804,
    -2.77008927719433, -2.77227214465915, 3.20203320709635, -2.89990738849473,
    -0.595221881324605, -4.28925378244217, 0.389724120274487, -1.14074717980657,
    -2.83005567450437, -0.0168224681808257, 0.416955705037907, 0.126910559061474,
    0.517232370861764, -0.0350493686052974, -0.465211076182388, 0.320198198514526,
    0.0445191674731724, 0.177839798284401, 0.261145002567677, -0.143275874698919,
    0.0
]

# Convert to numpy arrays
legit_array = np.array(legit_example).reshape(1, -1)
fraud_array = np.array(fraud_example).reshape(1, -1)

# Predictions
for name, example in [("Legitimate", legit_array), ("Fraudulent", fraud_array)]:
    pred = model.predict(example)[0]
    prob = model.predict_proba(example)[0][1]  # probability of fraud
    print(f"{name} Prediction: {'Fraudulent' if pred == 1 else 'Legitimate'} | Probability: {prob:.4f}")


Legitimate Prediction: Legitimate | Probability: 0.0001
Fraudulent Prediction: Fraudulent | Probability: 1.0000
